In [ ]:
import Pkg
Pkg.add(path="/home/tmb/proj/WebDataset.jl")

In [ ]:
using WebDataset

In [ ]:
using Transducers

In [ ]:
function stage(f::Function, rest...; csize=100, ntasks=1, verbose=false, kw...)
    outch = Channel(csize)
    return x -> begin
        if verbose; @show "starting", f, x, outch, rest; end
        Threads.@spawn f(x, outch, rest...; kw...)
        return outch
    end
end

In [ ]:
function repeatedly(shards, outch)
    while true
        for shard in shards
            put!(outch, shard)
        end
    end
end

function testch(inch, outch; name="testch")
    count = 0
    for sample in inch
        if count < 5;
            xsample = (string(sample)*" "^100)[1:100]
            @show name, xsample; 
        end
        put!(outch, sample)
        count += 1
    end
end

In [ ]:
function shard_reader(shards; nbuffer=10, nthreads=1)
    shard_ch = Channel(length(shards)+1)
    for shard in shards
        put!(shard_ch, shard)
    end
    sample_ch = Channel(nbuffer)
    threads = [Threads.@spawn tar_stage(shard_ch, sample_ch) for i in 1:nthreads]
    return sample_ch
end

In [ ]:
function preprocess_stage(inch, outch; nbuffer=10, nthreads=4, shuffle=0, decoders=default_decoders, augmenters=[])
    foreach(inch |> PartitionBy(itemkey) |> Map(transform↑(decoders)) |> Map(transform↑(augmenters))) do sample
    #foreach(inch) do sample
        put!(outch, sample)
    end
end

In [ ]:
ch = ["test.tar"] |> stage(repeatedly) |> stage(tar_stage; csize=200) |> stage(preprocess_stage; csize=100)

In [ ]:
inch = Channel(100)
put!(inch, "test.tar")
outch = Channel(100)
Threads.@spawn tar_stage(inch, outch; maxcount=99999999)
sleep(1)
outch

In [ ]:
inch = Channel(100)
put!(inch, "test.tar")
ch = inch |> stage(tar_stage; csize=200)
sleep(1)
ch

In [ ]:
inch = Channel(100)
put!(inch, "test.tar")
ch = inch |> stage(testch; name="in") |> stage(tar_stage; csize=200)
sleep(1)
ch

In [ ]:
inch = Channel(100)
put!(inch, "test.tar")
ch = ["test.tar"] |> stage(repeatedly) |> stage(testch; name="in") |> stage(tar_stage; csize=200) |> stage(testch; name="out")
sleep(1)
ch

In [ ]:
["test.tar"] |> stage(repeatedly) |> stage(tar_stage; csize=200) |> Take(5000) |> counted